In [ ]:
include("../analysis/mESC/load_analysis.jl");

# Fig. 2A: Mean expression

In [2]:
means_G1 = mean.(xG1)
means_G2M = mean.(xG2M)
means_ratio = means_G2M ./ means_G1;

In [ ]:
median(means_ratio)

In [ ]:
quantile(means_ratio, 0.25)

In [ ]:
quantile(means_ratio, 0.75)

In [ ]:
function stairpts(s)
    pts = s.plots[1].converted[1][]
    [p[1] for p in pts], [p[2] for p in pts]
end

In [ ]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*0.9), figure_padding = 1)
ax = Axis(f[1,1], xlabel="Ratio of means Λ", ylabel="Probability")

rmax = ceil(maximum(means_ratio), digits=1)
dr = 0.05
rs = 0:dr:rmax
ws = fit(Histogram, means_ratio, vcat(rs, rmax+dr), closed=:right)
ws = normalize(ws, mode=:probability)
ws = ProbabilityWeights(ws.weights)

ylims!(0, nothing)
xlims!(0.5, 2.5)

s = stairs!(ax, rs, ws, step=:post, color=(c1, 1), linewidth=0)

xs′, ys′ = stairpts(s)
band!(ax, xs′, 0*ys′, ys′, color=(c1, 1), label="Data")
vlines!(ax, 1, linewidth=1, color=(c3, 1), linestyle=:dash)
vlines!(ax, 2, linewidth=1, color=(c3, 1), linestyle=:dash)

In [ ]:
f

# Fig. 2B: Cartoon

In [ ]:
function plot_fit_new(f::GridPosition, xdata::AbstractArray, d::Distribution, nconv::Int; kwargs...)
    ax = Axis(f, 
          xlabel="Counts", ylabel="Probability", kwargs...)
    xs, ww = fit_hist(xdata)
    ww = ww ./ ww.sum
    
    ylims!(0, nothing)
    xlims!(-0.8, xs[end]+0.8)
    
    s = stairs!(ax, vcat(-1, xs, xs[end]+1), vcat(0, ww, 0), 
               step=:center, color=(c1, 1), linewidth=0)

    xs′, ys′ = stairpts(s)
    band!(ax, xs′, 0*ys′, ys′, color=(colorant"#d0d0d1", 1), label="Data")
    vlines!(ax, mean(xdata), linewidth=1, color=(c3, 0.8))
    lines!(ax, xs, convn(pdf.(d, xs), nconv), color=(c1, 1), linewidth=1, label="MLE fit")
    ax
end

function plot_fit_new(xdata::AbstractArray, d::Distribution, nconv::Int; kwargs...)
    f = Figure(size = (size_pt[1]*0.65, size_pt[2]*0.7), figure_padding = 3)
    plot_fit_new(f[1,1], xdata, d, nconv, kwargs...)
    f
end

In [ ]:
ind = findfirst(gene_names .== "Nme1")

In [11]:
d = G1_th_ind_fits[ind]
x = thetaG1
nconv=2
y = xG1[ind];

In [ ]:
plot_fit_new(y, d, nconv)

In [ ]:
f = Figure(size = (size_pt[1]*0.55, size_pt[2]*0.95), figure_padding = 1)
ga = f[1,1] = GridLayout()

xlim = 85
y = xG1[ind]
nconv = 2
d = G1_th_ind_fits[ind]

ax1 = plot_fit_new(ga[1,1], y, d, nconv)
ax1.xlabel = ""
ax1.xticklabelsvisible = false
xlims!(ax1, (0, xlim))

y = xG2M[ind]
nconv=4
d = G2M_th_ind_fits[ind]

ax2 = plot_fit_new(ga[2,1], y, d, nconv)
ax2.ylabel = ""
xlims!(ax2, (0, xlim))

rowgap!(ga, 5)

f

# Fig. 2C: example ratios + CIs

In [14]:
burst_freqs_G1_th_ind = get_burst_frequency.(G1_th_ind_fits)
burst_freqs_G2M_th_ind = get_burst_frequency.(G2M_th_ind_fits);

In [15]:
burst_sizes_G1_th_ind = get_burst_size.(G1_th_ind_fits)
burst_sizes_G2M_th_ind = get_burst_size.(G2M_th_ind_fits);

In [16]:
ratio_f = burst_freqs_G2M_th_ind ./ burst_freqs_G1_th_ind
ratio_b = burst_sizes_G2M_th_ind ./ burst_sizes_G1_th_ind;

In [17]:
genes = ["Avpi1", "Rpl7", "Akap9", "Smc1a", "Cobl"]
inds = [findfirst(gene_names .== gene) for gene in genes];

In [ ]:
println(means_ratio[inds])

In [ ]:
CI_ratio_f = @time get_ratio_confidence_intervals(G1_th_ind_fits[inds], xG1[inds], 2,
                                                  G2M_th_ind_fits[inds], xG2M[inds], 4,
                                                  get_burst_frequency_ratio_CI_PL, time_limit=60.0)

In [ ]:
CI_ratio_b = @time get_ratio_confidence_intervals(G1_th_ind_fits[inds], xG1[inds], 2,
                                                  G2M_th_ind_fits[inds], xG2M[inds], 4,
                                                  get_burst_size_ratio_CI_PL, time_limit=60.0)

In [21]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*1.35), figure_padding = 1)
ga = f[1,1] = GridLayout()

ax = Axis(ga[1,1],
          xlabel="", ylabel="Burst frequency ratio", xticklabelsvisible=false)
x = 1:length(genes)
ax.xticks = (x, genes)

y = ratio_f[inds]
y_lb = first.(CI_ratio_f)
y_ub = last.(CI_ratio_f)
scatter!(ax, x, y, markersize=7, color=c2)
errorbars!(ax, x, y, y .- y_lb, y_ub .- y, whiskerwidth=5, color=c2)
hlines!(ax, 1, color=(:black, 0.5), linestyle=:dash)
ylims!(ax, (0.3, 1.4))

ax = Axis(ga[2,1],
          xlabel="", ylabel="Burst size ratio")
x = 1:length(genes)
ax.xticks = (x, genes)

y = ratio_b[inds]
y_lb = first.(CI_ratio_b)
y_ub = last.(CI_ratio_b)
scatter!(ax, x, y, markersize=7, color=c1)
errorbars!(ax, x, y, y .- y_lb, y_ub .- y, whiskerwidth=5, color=c1)
hlines!(ax, 1, color=(:black, 0.5), linestyle=:dash)

rowgap!(ga, 3)

In [ ]:
f

# Fig. 2D: raincloud plots of ratios

In [ ]:
@show median(ratio_f)
@show quantile(ratio_f, 0.25)
@show quantile(ratio_f, 0.75);

In [ ]:
@show median(ratio_b)
@show quantile(ratio_b, 0.25)
@show quantile(ratio_b, 0.75);

In [ ]:
x = ratio_f
y = ratio_b
f = Figure(size = (size_pt[1]*0.85, size_pt[2]*1.0), figure_padding = 1)


axmain = Axis(f[1, 1], xlabel = "", ylabel = "",
              yminorticks = IntervalsBetween(2),
              yminorticksvisible = true,
              yminorticksize = 1.5,
              yminortickwidth = 0.7)

scatter!(axmain, x, y, color=(cx2, 0.4), markersize=3)
vlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
hlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
xlims!(axmain, low =0, high=1.5)
ylims!(axmain, low=0, high=3)
f

In [ ]:
f = Figure(size = (size_pt[1]*0.85, size_pt[2]*1.0), figure_padding = 1)
y1 = ratio_f; x1 = fill("f", length(y1))
y2 = ratio_b; x2 = fill("b", length(y2))
ax = Axis(f[1, 1], ylabel="", xlabel="", yticks=(1:2, ["f", "b"]))
rainclouds!(ax, vcat(x2, x1), vcat(y2, y1), gap=-1.3, 
            orientation = :horizontal,
            color = vcat(fill(c1, length(y1)), fill(c2, length(y2))),
            cloud_width=1.0, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.08, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.18, markersize=1.3, side_nudge=0.105)
vlines!(1, color=(:black, 0.4), linestyle=:dash)
xlims!(0, 2.5)
f